# Likelihood Serialization and Preservation


<p align="center">
<a href="https://inspirehep.net/record/1698425"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/arXiv_1810-05648_header.png"></a>
</p>
<p align="center">
<a href="https://twitter.com/lukasheinrich_/status/1052142936803160065"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/pyhf_arXiv.gif" width="300" height="214" border="0"></a>
</p>

<p align="center">
<a href="https://cds.cern.ch/record/2684863"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/ATLAS_PUB_Note_title.png"></a>
</p>
<p align="center">
<a href="https://home.cern/news/news/knowledge-sharing/new-open-release-allows-theorists-explore-lhc-data-new-way"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/CERN_news_story.png" border="0"></a>
</p>

## Preserved on HEPData

As of PyHEP 2020, ATLAS has published 4 full likelihoods to HEPData

<p align="center">
<a href="https://www.hepdata.net/record/ins1755298?version=2"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/HEPData_likelihoods.png"></a>
</p>

In [ ]:
import requests
import hashlib
import tarfile
import json

electroweakino_HEPData_URL = "https://www.hepdata.net/record/resource/1267798?view=true"
targz_filename = "1Lbb_workspaces.tar.gz"
response = requests.get(electroweakino_HEPData_URL, stream=True)
assert response.status_code == 200
with open(targz_filename, "wb") as file:
    file.write(response.content)
assert (
    hashlib.sha256(open(targz_filename, "rb").read()).hexdigest()
    == "64bbbef9f1aaf9e30d75c8975de4789484329b2b825d89331a6f2081661aa728"
)
hashlib.sha256(open(targz_filename, "rb").read()).hexdigest()
# Open as a tarfile
tar_archive = tarfile.open(targz_filename, "r:gz")

In [ ]:
def get_json_from_tarfile(tarfile, json_name):
    json_file = tarfile.extractfile(tarfile.getmember(json_name)).read().decode("utf8")
    return json.loads(json_file)


def get_bkg_and_signal(tarfile, directory_name, model_point):
    background_only = get_json_from_tarfile(tarfile, directory_name + "/BkgOnly.json",)
    patchset = pyhf.PatchSet(
        get_json_from_tarfile(tarfile, directory_name + "/patchset.json")
    )
    signal_patch = patchset[model_point]
    return background_only, signal_patch

In [ ]:
import pyhf

def calculate_CLs(bkgonly_json, signal_patch_json):
    """
    Calculate the observed CLs and the expected CLs band from a background only
    and signal patch.
    Args:
        bkgonly_json: The JSON for the background only model
        signal_patch_json: The JSON Patch for the signal model
    Returns:
        CLs_obs: The observed CLs value
        CLs_exp: List of the expected CLs value band
    """
    workspace = pyhf.workspace.Workspace(bkgonly_json)
    model = workspace.model(
        measurement_name=None,
        patches=[signal_patch_json],
        modifier_settings={
            "normsys": {"interpcode": "code4"},
            "histosys": {"interpcode": "code4p"},
        },
    )
    result = pyhf.infer.hypotest(
        1.0, workspace.data(model), model, qtilde=True, return_expected_set=True
    )
    return result[0].tolist()[0], result[-1].ravel().tolist()

In [ ]:
oneLbb_background, oneLbb_Wh_hbb_750_100_signal_patch = get_bkg_and_signal(
    tar_archive, "1Lbb-likelihoods-hepdata",(750, 100),  # "C1N2_Wh_hbb_750_100(750, 100)"
    )

In [ ]:
CLs_obs, CLs_exp_band = calculate_CLs(oneLbb_background, oneLbb_Wh_hbb_750_100_signal_patch)

In [ ]:
print('Observed: {}, Expected band: {}'.format(CLs_obs, CLs_exp_band))